In [1]:
from bs4 import BeautifulSoup
import glob
import os

courses_directory = 'nbgrader'
#course = ""
#assignment = ""

#feedback = os.path.join(courses_directory, course, 'feedback', '*', assignment, '*.html')
#feedback
result = ""

def getResult():
    global result
    return result

def setPath(c, a):
    course = c
    assignment = a
    feedback = os.path.join(courses_directory, course, 'feedback', '*', assignment, '*.html')
    return feedback
    #print(feedback)

def fetch(feedback):
    global result
    feedback_files = glob.glob(feedback)
    #feedback_files
    result = ""
    if (not feedback_files):
        result = "no feedback to summarize."
        return result
    
    for f in feedback_files:
        
        normalized_path = os.path.normpath(f)
        print(normalized_path)
        # Split the normalized path into its components
        parts = normalized_path.split(os.sep)  # os.sep is the appropriate separator
        index = parts.index('feedback')
        studentId = parts[index + 1]
        
        with open(normalized_path, 'r', encoding='utf-8') as file:
            html_content = file.read()
        
        soup = BeautifulSoup(html_content, 'html.parser')
        
        ph = soup.find('div', class_="panel-heading")
        overall = str(ph.find('h4')).split("Score:")[1].split(")")[0].strip()
        result += (studentId + "\n")
        result += (f"Overall Score: {overall}\n")
    
        qs = ph.find('ol').find_all('li')
        count = 1
        for q in qs:
            href = str(q)[str(q).find('#')+1:str(q).find('">')]
            t = BeautifulSoup(str(q), 'html.parser').get_text()
            
            if (t == "Comment"):
                anchor = soup.find('a', attrs={'name': href})
                div = anchor.find_parent('div')
                comment = BeautifulSoup(str(div.find('div', class_="panel-footer").find('div').get_text()), 'html.parser')
                result += (comment.get_text().strip() + "\n")
            else:
                result += (str(count) + ". " + t + "\n")
                count += 1   
            
        result += ("--------------------------\n")
    return result

In [18]:
import ipywidgets as widgets
from IPython.display import display
import os
import threading
import time

op = None
th = 0

course = widgets.Dropdown(
    options=[],
    description='Course:',
    disabled=False
)

assignment = widgets.Dropdown(
    options=[],
    description='Assignment:',
    disabled=False
)

button = widgets.Button(
    description='Submit',
    layout=widgets.Layout(margin='0px 0px 0px 90px'), 
    disabled=True
)

textbox = widgets.Textarea(
    description='Output:',
    layout=widgets.Layout(width='50%', height='300px'),  # Set width and height
    disabled=True
)

pathText = widgets.Text(
    description="Current Path:",
    value=courses_directory,
    disabled=True
)

pathUpdate = widgets.Text(
    description="Update Path:",
)

btnUpdate = widgets.Button(
    description='Update',
    layout=widgets.Layout(margin='0px 0px 0px 90px'),
)

title = widgets.HTML(
    value="<center><h1>AI Summarizing</h1></center>"
)

grid = widgets.GridspecLayout(3, 2, width='50%')
grid[0,0] = course
grid[1,0] = assignment
grid[2,0] = button
grid[0,1] = pathText
grid[1,1] = pathUpdate
grid[2,1] = btnUpdate

box_layout = widgets.Layout(display='flex',
                    flex_flow='column', 
                    align_items='center',
                    justify_content='center',
                    width='100%',
                    height='100%',
                    margin='250px 0px 0px 0px')

box = widgets.Box(children=[title, grid, textbox], layout=box_layout)

def course_initilize():
    courses = [name for name in os.listdir(courses_directory) if os.path.isdir(os.path.join(courses_directory, name))]
    courses.insert(0, "---SELECT COURSE---")
    course.options = courses
    course.value = course.options[0]
    
def course_change(change):
    selected_course = change['new']
    if (selected_course == "---SELECT COURSE---"):
        assignment.options = []  # Clear options if no assignments exist
        assignment.value = None
    else:
        assignment_path = os.path.join(courses_directory, selected_course, 'source')
        if os.path.exists(assignment_path):
            assignments = [name for name in os.listdir(assignment_path) if os.path.isdir(os.path.join(assignment_path, name))]
            assignment.options = assignments  # Update assignment options
            assignment.value = assignments[0]
        else:
            assignment.options = []  # Clear options if no assignments exist
            assignment.value = None

def countTime():
    global op
    global th
    i = 0
    th = 0
    dot = [".", "..", "..."]
    while (op is None):
        textbox.value = "Executing" + dot[i%3] + "\n" + str(i) + " seconds"
        time.sleep(1)
        i += 1
    th = i

def assignment_change(change):
    if assignment.value is not None:
        button.disabled = False
    else:
        button.disabled = True

def on_button_clicked(b):
    global op
    global th
    f = setPath(course.value, assignment.value)
    t = fetch(f)
    if (t == "no feedback to summarize."):
        textbox.value = t
    elif (t):
        textbox.value = "Executing..."
        button.disabled = True
        op = None
        threading.Thread(target=countTime).start()
        %run -i summarize_teacher.ipynb
        op = summarize(t)
        time.sleep(1)
        textbox.value = op + "\nExecuted for " + str(th) + " seconds"
        button.disabled = False

def on_btnUpdate_clicked(b):
    global courses_directory
    if os.path.isdir(pathUpdate.value):
        courses_directory = pathUpdate.value
        pathText.value = courses_directory
        course_initilize()
        if textbox.value == "Invalid path":
            textbox.value = ""
    else:
        textbox.value = "Invalid path"
    
course_initilize()
course.observe(course_change, names='value')
assignment.observe(assignment_change, names='value')
button.on_click(on_button_clicked)
btnUpdate.on_click(on_btnUpdate_clicked)

display(box)

Box(children=(HTML(value='<center><h1>AI Summarizing</h1></center>'), GridspecLayout(children=(Dropdown(descri…